In [1]:
import pyzx as zx
import random
from qiskit import QuantumCircuit
from qiskit.circuit import QuantumCircuit, ParameterVector
import csv
import time
import japanize_matplotlib # 追加

In [2]:
def generate_H_S_CNOT_T_circuit(qubits, gates, p_t=0, seed=1000):
    random.seed(seed)  
    p_s = 0.333 * (1.0 - p_t)  
    p_had = 0.333 * (1.0 - p_t)  
    p_cnot = 0.333 * (1.0 - p_t)  

    c = zx.Circuit(qubits) 
    for _ in range(gates):
        r = random.random() 
        if r < p_had:
            c.add_gate("HAD", random.randrange(qubits))
        elif r < p_had + p_s:
            c.add_gate("S", random.randrange(qubits))
        elif r < p_had + p_s + p_t:
            c.add_gate("T", random.randrange(qubits))
        else:
            tgt = random.randrange(qubits)
            while True:
                ctrl = random.randrange(qubits)
                if ctrl != tgt:
                    break
            c.add_gate("CNOT", tgt, ctrl)
    return c

In [ ]:

c = generate_H_S_CNOT_T_circuit(4, 50, p_t=0.4, seed=1000)

c2 = c.copy()
zx.draw(c2.to_graph())
g = c.to_graph()
zx.teleport_reduce(g)
c3 = zx.Circuit.from_graph(g)
zx.draw(c3.to_graph())

#c4 = zx.optimize.full_optimize(c3)
c4 = zx.optimize.basic_optimization(c3)
zx.draw(c4.to_graph())
c5 = zx.optimize.full_optimize(c3)
zx.draw(c5.to_graph())


In [57]:
c = generate_H_S_CNOT_T_circuit(4, 1000, p_t=0.4, seed=1000)

print("original")
print(c.stats_dict())

print("after pre-processing_full")
#c = zx.optimize.basic_optimization(c)
c = zx.optimize.full_optimize(c)
print(c.stats_dict())
g = c.to_graph()


print("after teleportation")
zx.teleport_reduce(g)
c = zx.Circuit.from_graph(g)
zx.draw(c.to_graph())
print(c.stats_dict())

c = zx.optimize.basic_optimization(c.copy())
print("basic")
zx.draw(c.to_graph()) 
print(c.stats_dict())

print("full")
c = zx.optimize.full_optimize(c.copy())
print(c.stats_dict())

original
{'name': '', 'qubits': 4, 'gates': 1000, 'tcount': 425, 'clifford': 575, 'twoqubit': 207, 'cnot': 207, 'had': 177, 'measurement': 0, 'other': 0, 'depth': 0, 'depth_cz': 0}
after pre-processing_full
{'name': '', 'qubits': 4, 'gates': 530, 'tcount': 117, 'clifford': 413, 'twoqubit': 247, 'cnot': 214, 'had': 99, 'measurement': 0, 'other': 0, 'depth': 0, 'depth_cz': 0}
after teleportation


{'name': '', 'qubits': 4, 'gates': 582, 'tcount': 117, 'clifford': 465, 'twoqubit': 247, 'cnot': 214, 'had': 99, 'measurement': 0, 'other': 0, 'depth': 0, 'depth_cz': 0}
basic


{'name': '', 'qubits': 4, 'gates': 530, 'tcount': 117, 'clifford': 413, 'twoqubit': 247, 'cnot': 214, 'had': 99, 'measurement': 0, 'other': 0, 'depth': 0, 'depth_cz': 0}
full
{'name': '', 'qubits': 4, 'gates': 508, 'tcount': 117, 'clifford': 391, 'twoqubit': 242, 'cnot': 206, 'had': 97, 'measurement': 0, 'other': 0, 'depth': 0, 'depth_cz': 0}


In [28]:
# seed固定
seeds_list = [1000,1001,1002,1003,1004]

# パラメータ設定
qubits_list = [4, 8, 12, 16]
p_t_lists = [i * 0.04 for i in range(0, 11)]
gate_num_lists = [10000]
optimize_list = ["original","basic","clifford","full","phase"]

# CSV書き込みの準備
output_file = "/home/rest/forpyzx/output/0422_teleport_basic_10000.csv"
fieldnames = [
    "optimize_name","qubit","p_t","input_gates","seed","depth_bef","all_gates","one-qubit_gates","two-qubit gates","t","H","otherRZ","CNOT","CZ","depth_aft","time"
]
start_all = time.time()
with open(output_file, "w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    for optimize_name in optimize_list:
        for qubit in qubits_list:
            for gate_num in gate_num_lists:
                for p_t in p_t_lists:
                    for seed in seeds_list:
                        try:
                            start = time.time()
                            # 回路作成
                            c = generate_H_S_CNOT_T_circuit(qubits = qubit,gates= gate_num,p_t = p_t,seed = seed)

                            depth_bef = QuantumCircuit.from_qasm_str(c.to_qasm()).depth()

                            if optimize_name == "basic":
                                c = zx.optimize.basic_optimization(c)
                            elif optimize_name == "clifford":
                                c = zx.optimize.basic_optimization(c)
                                g = c.to_graph()
                                zx.simplify.clifford_simp(g)
                                c = zx.extract.extract_circuit(g, up_to_perm=True)
                                c = zx.optimize.basic_optimization(c)
                            elif optimize_name == "original":
                                pass
                            elif optimize_name == "full":
                                c = zx.optimize.basic_optimization(c)
                                g = c.to_graph()
                                zx.simplify.full_reduce(g)
                                c = zx.extract.extract_circuit(g, up_to_perm=True)
                                c = zx.optimize.basic_optimization(c)
                            elif optimize_name == "phase":
                                c = zx.optimize.basic_optimization(c)
                                g = c.to_graph()
                                zx.teleport_reduce(g)
                                c = zx.Circuit.from_graph(g)
                                c = zx.optimize.basic_optimization(c)
                            else:
                                raise ValueError(f"Unknown optimization name: {optimize_name}")

                            end = time.time()

                            #qiskit変換とカウント
                            circuit_qiskit = QuantumCircuit.from_qasm_str(c.to_basic_gates().to_qasm())
                            depth_aft = circuit_qiskit.depth()
                            zx_dict = c.to_basic_gates().stats_dict()
                            qiskit_dict = circuit_qiskit.count_ops()

                            gates_aft = zx_dict["gates"]


                            # 結果をCSVに書き込む
                            writer.writerow({
                                "optimize_name": optimize_name,
                                "qubit": qubit,
                                "p_t": p_t,
                                "input_gates": gate_num,
                                "seed": seed,
                                "depth_bef": depth_bef,
                                "all_gates": gates_aft,
                                "depth_aft": depth_aft,
                                "t": zx_dict["tcount"],
                                "H": qiskit_dict.get("h", 0),
                                "otherRZ": gates_aft - qiskit_dict.get("h", 0) - qiskit_dict.get("cx", 0) - qiskit_dict.get("cz", 0) - zx_dict["tcount"],
                                "one-qubit_gates": gates_aft - qiskit_dict.get("cx", 0) - qiskit_dict.get("cz", 0),
                                "two-qubit gates": qiskit_dict.get("cx", 0) + qiskit_dict.get("cz", 0),
                                "CNOT": qiskit_dict.get("cx", 0),
                                "CZ": qiskit_dict.get("cz", 0),
                                "time": end - start
                            })

                            print(f"done. Qubits={qubit}, Gate_Num={gate_num}, P_T={p_t},seed={seed},time={end-start}")

                        except Exception as e:
                            print(f"Error for Qubits={qubit}, Gate_Num={gate_num}, P_T={p_t}: {e}")

end_all = time.time()
print(f"Total time: {end_all - start_all}")

done. Qubits=4, Gate_Num=10000, P_T=0.0,seed=1000,time=0.0965888500213623
done. Qubits=4, Gate_Num=10000, P_T=0.0,seed=1001,time=0.055449724197387695
done. Qubits=4, Gate_Num=10000, P_T=0.0,seed=1002,time=0.05788373947143555
done. Qubits=4, Gate_Num=10000, P_T=0.0,seed=1003,time=0.05589795112609863
done. Qubits=4, Gate_Num=10000, P_T=0.0,seed=1004,time=0.09481072425842285
done. Qubits=4, Gate_Num=10000, P_T=0.04,seed=1000,time=0.05766773223876953
done. Qubits=4, Gate_Num=10000, P_T=0.04,seed=1001,time=0.05702829360961914
done. Qubits=4, Gate_Num=10000, P_T=0.04,seed=1002,time=0.057504892349243164
done. Qubits=4, Gate_Num=10000, P_T=0.04,seed=1003,time=0.09051942825317383
done. Qubits=4, Gate_Num=10000, P_T=0.04,seed=1004,time=0.05684685707092285
done. Qubits=4, Gate_Num=10000, P_T=0.08,seed=1000,time=0.0608220100402832
done. Qubits=4, Gate_Num=10000, P_T=0.08,seed=1001,time=0.05581092834472656
done. Qubits=4, Gate_Num=10000, P_T=0.08,seed=1002,time=0.10241842269897461
done. Qubits=4, G

In [ ]:
#teleport　後処理あり
# seed固定
seeds_list = [1000,1001,1002,1003,1004]

# パラメータ設定
qubits_list = [4, 8, 12, 16]
p_t_lists = [i * 0.04 for i in range(0, 11)]
gate_num_lists = [10000]
optimize_list = ["phase"]

# CSV書き込みの準備
output_file = "/home/rest/forpyzx/output/0422_only_teleport_basic_10000.csv"
fieldnames = [
    "optimize_name","qubit","p_t","input_gates","seed","depth_bef","all_gates","one-qubit_gates","two-qubit gates","t","H","otherRZ","CNOT","CZ","depth_aft","time"
]
start_all = time.time()
with open(output_file, "w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    for optimize_name in optimize_list:
        for qubit in qubits_list:
            for gate_num in gate_num_lists:
                for p_t in p_t_lists:
                    for seed in seeds_list:
                        try:
                            start = time.time()
                            # 回路作成
                            c = generate_H_S_CNOT_T_circuit(qubits = qubit,gates= gate_num,p_t = p_t,seed = seed)

                            depth_bef = QuantumCircuit.from_qasm_str(c.to_qasm()).depth()

                            if optimize_name == "phase":
                                c = zx.optimize.basic_optimization(c)
                                g = c.to_graph()
                                c = zx.Circuit.from_graph(zx.teleport_reduce(g))
                                c = zx.optimize.basic_optimization(c)
                            else:
                                raise ValueError(f"Unknown optimization name: {optimize_name}")

                            end = time.time()

                            #qiskit変換とカウント
                            circuit_qiskit = QuantumCircuit.from_qasm_str(c.to_basic_gates().to_qasm())
                            depth_aft = circuit_qiskit.depth()
                            zx_dict = c.to_basic_gates().stats_dict()
                            qiskit_dict = circuit_qiskit.count_ops()

                            gates_aft = zx_dict["gates"]
                            # 結果をCSVに書き込む
                            writer.writerow({
                                "optimize_name": optimize_name,
                                "qubit": qubit,
                                "p_t": p_t,
                                "input_gates": gate_num,
                                "seed": seed,
                                "depth_bef": depth_bef,
                                "all_gates": gates_aft,
                                "depth_aft": depth_aft,
                                "t": zx_dict["tcount"],
                                "H": qiskit_dict.get("h", 0),
                                "otherRZ": gates_aft - qiskit_dict.get("h", 0) - qiskit_dict.get("cx", 0) - qiskit_dict.get("cz", 0) - zx_dict["tcount"],
                                "one-qubit_gates": gates_aft - qiskit_dict.get("cx", 0) - qiskit_dict.get("cz", 0),
                                "two-qubit gates": qiskit_dict.get("cx", 0) + qiskit_dict.get("cz", 0),
                                "CNOT": qiskit_dict.get("cx", 0),
                                "CZ": qiskit_dict.get("cz", 0),
                                "time": end - start
                            })

                            print(f"done. Qubits={qubit}, Gate_Num={gate_num}, P_T={p_t},seed={seed},time={end-start}")

                        except Exception as e:
                            print(f"Error for Qubits={qubit}, Gate_Num={gate_num}, P_T={p_t}: {e}")

end_all = time.time()
print(f"Total time: {end_all - start_all}")

done. Qubits=4, Gate_Num=10000, P_T=0.0,seed=1000,time=2.3582656383514404
done. Qubits=4, Gate_Num=10000, P_T=0.0,seed=1001,time=2.2062604427337646
done. Qubits=4, Gate_Num=10000, P_T=0.0,seed=1002,time=2.394876480102539
done. Qubits=4, Gate_Num=10000, P_T=0.0,seed=1003,time=2.149045705795288
done. Qubits=4, Gate_Num=10000, P_T=0.0,seed=1004,time=2.445148229598999
done. Qubits=4, Gate_Num=10000, P_T=0.04,seed=1000,time=2.381479501724243
done. Qubits=4, Gate_Num=10000, P_T=0.04,seed=1001,time=2.356309652328491
done. Qubits=4, Gate_Num=10000, P_T=0.04,seed=1002,time=2.8221042156219482
done. Qubits=4, Gate_Num=10000, P_T=0.04,seed=1003,time=2.622382879257202
done. Qubits=4, Gate_Num=10000, P_T=0.04,seed=1004,time=2.6321535110473633
done. Qubits=4, Gate_Num=10000, P_T=0.08,seed=1000,time=2.4905121326446533
done. Qubits=4, Gate_Num=10000, P_T=0.08,seed=1001,time=2.5869107246398926
done. Qubits=4, Gate_Num=10000, P_T=0.08,seed=1002,time=2.541957139968872
done. Qubits=4, Gate_Num=10000, P_T=0

In [64]:
#teleport　後処理あり 前処理無し
# seed固定
seeds_list = [1000,1001,1002,1003,1004]

# パラメータ設定
qubits_list = [4, 8, 12, 16]
p_t_lists = [i * 0.04 for i in range(0, 11)]
gate_num_lists = [10000]
optimize_list = ["phase_no_pre"]

# CSV書き込みの準備
output_file = "/home/rest/forpyzx/output/0422_nopre_teleport_basic_10000.csv"
fieldnames = [
    "optimize_name","qubit","p_t","input_gates","seed","depth_bef","all_gates","one-qubit_gates","two-qubit gates","t","H","otherRZ","CNOT","CZ","depth_aft","time"
]
start_all = time.time()
with open(output_file, "w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    for optimize_name in optimize_list:
        for qubit in qubits_list:
            for gate_num in gate_num_lists:
                for p_t in p_t_lists:
                    for seed in seeds_list:
                        try:
                            start = time.time()
                            # 回路作成
                            c = generate_H_S_CNOT_T_circuit(qubits = qubit,gates= gate_num,p_t = p_t,seed = seed)

                            depth_bef = QuantumCircuit.from_qasm_str(c.to_qasm()).depth()

                            if optimize_name == "phase_no_pre":
                                #c = zx.optimize.basic_optimization(c)
                                g = c.to_graph()
                                c = zx.Circuit.from_graph(zx.teleport_reduce(g))
                                c = zx.optimize.basic_optimization(c)
                            else:
                                raise ValueError(f"Unknown optimization name: {optimize_name}")

                            end = time.time()

                            #qiskit変換とカウント
                            circuit_qiskit = QuantumCircuit.from_qasm_str(c.to_basic_gates().to_qasm())
                            depth_aft = circuit_qiskit.depth()
                            zx_dict = c.to_basic_gates().stats_dict()
                            qiskit_dict = circuit_qiskit.count_ops()

                            gates_aft = zx_dict["gates"]
                            # 結果をCSVに書き込む
                            writer.writerow({
                                "optimize_name": optimize_name,
                                "qubit": qubit,
                                "p_t": p_t,
                                "input_gates": gate_num,
                                "seed": seed,
                                "depth_bef": depth_bef,
                                "all_gates": gates_aft,
                                "depth_aft": depth_aft,
                                "t": zx_dict["tcount"],
                                "H": qiskit_dict.get("h", 0),
                                "otherRZ": gates_aft - qiskit_dict.get("h", 0) - qiskit_dict.get("cx", 0) - qiskit_dict.get("cz", 0) - zx_dict["tcount"],
                                "one-qubit_gates": gates_aft - qiskit_dict.get("cx", 0) - qiskit_dict.get("cz", 0),
                                "two-qubit gates": qiskit_dict.get("cx", 0) + qiskit_dict.get("cz", 0),
                                "CNOT": qiskit_dict.get("cx", 0),
                                "CZ": qiskit_dict.get("cz", 0),
                                "time": end - start
                            })

                            print(f"done. Qubits={qubit}, Gate_Num={gate_num}, P_T={p_t},seed={seed},time={end-start}")

                        except Exception as e:
                            print(f"Error for Qubits={qubit}, Gate_Num={gate_num}, P_T={p_t}: {e}")

end_all = time.time()
print(f"Total time: {end_all - start_all}")

done. Qubits=4, Gate_Num=10000, P_T=0.0,seed=1000,time=3.6362240314483643
done. Qubits=4, Gate_Num=10000, P_T=0.0,seed=1001,time=3.3477835655212402
done. Qubits=4, Gate_Num=10000, P_T=0.0,seed=1002,time=3.597249746322632
done. Qubits=4, Gate_Num=10000, P_T=0.0,seed=1003,time=3.444125175476074
done. Qubits=4, Gate_Num=10000, P_T=0.0,seed=1004,time=3.500577211380005
done. Qubits=4, Gate_Num=10000, P_T=0.04,seed=1000,time=3.3893325328826904
done. Qubits=4, Gate_Num=10000, P_T=0.04,seed=1001,time=3.73073148727417
done. Qubits=4, Gate_Num=10000, P_T=0.04,seed=1002,time=3.4115467071533203
done. Qubits=4, Gate_Num=10000, P_T=0.04,seed=1003,time=3.6008479595184326
done. Qubits=4, Gate_Num=10000, P_T=0.04,seed=1004,time=3.5634310245513916
done. Qubits=4, Gate_Num=10000, P_T=0.08,seed=1000,time=3.560088634490967
done. Qubits=4, Gate_Num=10000, P_T=0.08,seed=1001,time=3.707404613494873
done. Qubits=4, Gate_Num=10000, P_T=0.08,seed=1002,time=3.4132206439971924
done. Qubits=4, Gate_Num=10000, P_T=0

In [5]:
#teleport　前後処理にfull 前処理ありなしバージョン
# seed固定
seeds_list = [1000,1001,1002,1003,1004]

# パラメータ設定
qubits_list = [4, 8, 12, 16]
p_t_lists = [i * 0.04 for i in range(0, 11)]
gate_num_lists = [10000]
optimize_list = ["phase_full","phase_full_no_pre"]

# CSV書き込みの準備
output_file = "/home/rest/forpyzx/output/0422_phasetele_full_10000.csv"
fieldnames = [
    "optimize_name","qubit","p_t","input_gates","seed","depth_bef","all_gates","one-qubit_gates","two-qubit gates","t","H","otherRZ","CNOT","CZ","depth_aft","time"
]
start_all = time.time()
with open(output_file, "w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    for optimize_name in optimize_list:
        for qubit in qubits_list:
            for gate_num in gate_num_lists:
                for p_t in p_t_lists:
                    for seed in seeds_list:
                        try:
                            start = time.time()
                            # 回路作成
                            c = generate_H_S_CNOT_T_circuit(qubits = qubit,gates= gate_num,p_t = p_t,seed = seed)

                            depth_bef = QuantumCircuit.from_qasm_str(c.to_qasm()).depth()

                            if optimize_name == "phase_no_pre":
                                #c = zx.optimize.basic_optimization(c)
                                g = c.to_graph()
                                c = zx.Circuit.from_graph(zx.teleport_reduce(g))
                                c = zx.optimize.full_optimize(c)
                            elif optimize_name == "phase_full":
                                c = zx.optimize.full_optimize(c)
                                g = c.to_graph()
                                c = zx.Circuit.from_graph(zx.teleport_reduce(g))
                                c = zx.optimize.full_optimize(c)
                            else:
                                raise ValueError(f"Unknown optimization name: {optimize_name}")

                            end = time.time()

                            #qiskit変換とカウント
                            circuit_qiskit = QuantumCircuit.from_qasm_str(c.to_basic_gates().to_qasm())
                            depth_aft = circuit_qiskit.depth()
                            zx_dict = c.to_basic_gates().stats_dict()
                            qiskit_dict = circuit_qiskit.count_ops()

                            gates_aft = zx_dict["gates"]
                            # 結果をCSVに書き込む
                            writer.writerow({
                                "optimize_name": optimize_name,
                                "qubit": qubit,
                                "p_t": p_t,
                                "input_gates": gate_num,
                                "seed": seed,
                                "depth_bef": depth_bef,
                                "all_gates": gates_aft,
                                "depth_aft": depth_aft,
                                "t": zx_dict["tcount"],
                                "H": qiskit_dict.get("h", 0),
                                "otherRZ": gates_aft - qiskit_dict.get("h", 0) - qiskit_dict.get("cx", 0) - qiskit_dict.get("cz", 0) - zx_dict["tcount"],
                                "one-qubit_gates": gates_aft - qiskit_dict.get("cx", 0) - qiskit_dict.get("cz", 0),
                                "two-qubit gates": qiskit_dict.get("cx", 0) + qiskit_dict.get("cz", 0),
                                "CNOT": qiskit_dict.get("cx", 0),
                                "CZ": qiskit_dict.get("cz", 0),
                                "time": end - start
                            })

                            print(f"done. Qubits={qubit}, Gate_Num={gate_num}, P_T={p_t},seed={seed},time={end-start}")

                        except Exception as e:
                            print(f"Error for Qubits={qubit}, Gate_Num={gate_num}, P_T={p_t}: {e}")

end_all = time.time()
print(f"Total time: {end_all - start_all}")

done. Qubits=4, Gate_Num=10000, P_T=0.0,seed=1000,time=3.89357852935791
done. Qubits=4, Gate_Num=10000, P_T=0.0,seed=1001,time=4.4942615032196045
done. Qubits=4, Gate_Num=10000, P_T=0.0,seed=1002,time=5.897325754165649
done. Qubits=4, Gate_Num=10000, P_T=0.0,seed=1003,time=6.191803455352783
done. Qubits=4, Gate_Num=10000, P_T=0.0,seed=1004,time=7.865337610244751
done. Qubits=4, Gate_Num=10000, P_T=0.04,seed=1000,time=10.154003620147705
done. Qubits=4, Gate_Num=10000, P_T=0.04,seed=1001,time=12.08194613456726
done. Qubits=4, Gate_Num=10000, P_T=0.04,seed=1002,time=6.010730028152466
done. Qubits=4, Gate_Num=10000, P_T=0.04,seed=1003,time=5.335028409957886
done. Qubits=4, Gate_Num=10000, P_T=0.04,seed=1004,time=6.880026578903198
done. Qubits=4, Gate_Num=10000, P_T=0.08,seed=1000,time=5.912743806838989
done. Qubits=4, Gate_Num=10000, P_T=0.08,seed=1001,time=6.541682243347168
done. Qubits=4, Gate_Num=10000, P_T=0.08,seed=1002,time=6.463658094406128
done. Qubits=4, Gate_Num=10000, P_T=0.08,s

KeyboardInterrupt: 

In [ ]:
#teleport　後処理無し
# seed固定
seeds_list = [1000,1001,1002,1003,1004]

# パラメータ設定
qubits_list = [4, 8, 12, 16]
p_t_lists = [i * 0.04 for i in range(0, 11)]
gate_num_lists = [10000]
optimize_list = ["phase_no_preaft"]

# CSV書き込みの準備
output_file = "/home/rest/forpyzx/output/0422_phasetele_no_preaft_10000.csv"
fieldnames = [
    "optimize_name","qubit","p_t","input_gates","seed","depth_bef","all_gates","one-qubit_gates","two-qubit gates","t","H","otherRZ","CNOT","CZ","depth_aft","time"
]
start_all = time.time()
with open(output_file, "w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    for optimize_name in optimize_list:
        for qubit in qubits_list:
            for gate_num in gate_num_lists:
                for p_t in p_t_lists:
                    for seed in seeds_list:
                        try:
                            start = time.time()
                            # 回路作成
                            c = generate_H_S_CNOT_T_circuit(qubits = qubit,gates= gate_num,p_t = p_t,seed = seed)

                            depth_bef = QuantumCircuit.from_qasm_str(c.to_qasm()).depth()

                            if optimize_name == "phase_no_preaft":
                                #c = zx.optimize.basic_optimization(c)
                                g = c.to_graph()
                                c = zx.Circuit.from_graph(zx.teleport_reduce(g))
                                #c = zx.optimize.full_optimize(c)
                            else:
                                raise ValueError(f"Unknown optimization name: {optimize_name}")

                            end = time.time()

                            #qiskit変換とカウント
                            circuit_qiskit = QuantumCircuit.from_qasm_str(c.to_basic_gates().to_qasm())
                            depth_aft = circuit_qiskit.depth()
                            zx_dict = c.to_basic_gates().stats_dict()
                            qiskit_dict = circuit_qiskit.count_ops()

                            gates_aft = zx_dict["gates"]
                            # 結果をCSVに書き込む
                            writer.writerow({
                                "optimize_name": optimize_name,
                                "qubit": qubit,
                                "p_t": p_t,
                                "input_gates": gate_num,
                                "seed": seed,
                                "depth_bef": depth_bef,
                                "all_gates": gates_aft,
                                "depth_aft": depth_aft,
                                "t": zx_dict["tcount"],
                                "H": qiskit_dict.get("h", 0),
                                "otherRZ": gates_aft - qiskit_dict.get("h", 0) - qiskit_dict.get("cx", 0) - qiskit_dict.get("cz", 0) - zx_dict["tcount"],
                                "one-qubit_gates": gates_aft - qiskit_dict.get("cx", 0) - qiskit_dict.get("cz", 0),
                                "two-qubit gates": qiskit_dict.get("cx", 0) + qiskit_dict.get("cz", 0),
                                "CNOT": qiskit_dict.get("cx", 0),
                                "CZ": qiskit_dict.get("cz", 0),
                                "time": end - start
                            })

                            print(f"done. Qubits={qubit}, Gate_Num={gate_num}, P_T={p_t},seed={seed},time={end-start}")

                        except Exception as e:
                            print(f"Error for Qubits={qubit}, Gate_Num={gate_num}, P_T={p_t}: {e}")

end_all = time.time()
print(f"Total time: {end_all - start_all}")

In [ ]:
#extractの性能比較

# seed固定
seeds_list = [1000,1001,1002]

# パラメータ設定
qubits_list = [4, 8, 12, 16]
p_t_lists = [i * 0.04 for i in range(0, 11)]
gate_num_lists = [5000]
#optimize_list = ["original","basic","clifford","full","clifford_look"]
#optimize_list = ["original","basic","clifford","full"]
optimize_list = ["clifford_look"]

# CSV書き込みの準備
output_file = "/home/rest/forpyzx/output/0527_cliffordlookforward_10000.csv"
fieldnames = [
    "optimize_name","qubit","p_t","input_gates","seed","depth_bef","all_gates","one-qubit_gates","two-qubit gates","t","H","otherRZ","CNOT","CZ","depth_aft","time"
]
start_all = time.time()
with open(output_file, "w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    for optimize_name in optimize_list:
        for qubit in qubits_list:
            for gate_num in gate_num_lists:
                for p_t in p_t_lists:
                    for seed in seeds_list:
                        try:
                            start = time.time()
                            # 回路作成
                            c = generate_H_S_CNOT_T_circuit(qubits = qubit,gates= gate_num,p_t = p_t,seed = seed)

                            depth_bef = QuantumCircuit.from_qasm_str(c.to_qasm()).depth()

                            if optimize_name == "basic":
                                c = zx.optimize.basic_optimization(c)
                            elif optimize_name == "clifford":
                                c = zx.optimize.basic_optimization(c)
                                g = c.to_graph()
                                zx.simplify.clifford_simp(g)
                                c = zx.extract.extract_circuit(g, up_to_perm=True)
                                c = zx.optimize.basic_optimization(c)
                            elif optimize_name == "original":
                                pass
                            elif optimize_name == "full":
                                c = zx.optimize.basic_optimization(c)
                                g = c.to_graph()
                                zx.simplify.full_reduce(g)
                                c = zx.extract.extract_circuit(g, up_to_perm=True)
                                c = zx.optimize.basic_optimization(c)
                            elif optimize_name == "full_look":
                                c = zx.optimize.basic_optimization(c)
                                g = c.to_graph()
                                zx.simplify.full_reduce(g)
                                c = zx.extract.lookahead_extract(g, up_to_perm=True)
                                c = zx.optimize.basic_optimization(c)
                            elif optimize_name == "clifford_look":
                                c = zx.optimize.basic_optimization(c)
                                g = c.to_graph()
                                zx.simplify.clifford_simp(g)
                                c = zx.extract.lookahead_extract(g, up_to_perm=True)
                                c = zx.optimize.basic_optimization(c)
                            else:
                                raise ValueError(f"Unknown optimization name: {optimize_name}")

                            end = time.time()

                            #qiskit変換とカウント
                            circuit_qiskit = QuantumCircuit.from_qasm_str(c.to_basic_gates().to_qasm())
                            depth_aft = circuit_qiskit.depth()
                            zx_dict = c.to_basic_gates().stats_dict()
                            qiskit_dict = circuit_qiskit.count_ops()
                            gates_aft = zx_dict["gates"]


                            # 結果をCSVに書き込む
                            writer.writerow({
                                "optimize_name": optimize_name,
                                "qubit": qubit,
                                "p_t": p_t,
                                "input_gates": gate_num,
                                "seed": seed,
                                "depth_bef": depth_bef,
                                "all_gates": gates_aft,
                                "depth_aft": depth_aft,
                                "t": zx_dict["tcount"],
                                "H": qiskit_dict.get("h", 0),
                                "otherRZ": gates_aft - qiskit_dict.get("h", 0) - qiskit_dict.get("cx", 0) - qiskit_dict.get("cz", 0) - zx_dict["tcount"],
                                "one-qubit_gates": gates_aft - qiskit_dict.get("cx", 0) - qiskit_dict.get("cz", 0),
                                "two-qubit gates": qiskit_dict.get("cx", 0) + qiskit_dict.get("cz", 0),
                                "CNOT": qiskit_dict.get("cx", 0),
                                "CZ": qiskit_dict.get("cz", 0),
                                "time": end - start
                            })

                            print(f"done. name={optimize_name},Qubits={qubit}, Gate_Num={gate_num}, P_T={p_t},seed={seed},time={end-start}")

                        except Exception as e:
                            print(f"Error for Qubits={qubit}, Gate_Num={gate_num}, P_T={p_t}: {e}")

end_all = time.time()
print(f"Total time: {end_all - start_all}")

done. name=original,Qubits=4, Gate_Num=5000, P_T=0.0,seed=1000,time=0.033101558685302734
done. name=original,Qubits=4, Gate_Num=5000, P_T=0.0,seed=1001,time=0.029905080795288086
done. name=original,Qubits=4, Gate_Num=5000, P_T=0.0,seed=1002,time=0.02855658531188965
done. name=original,Qubits=4, Gate_Num=5000, P_T=0.04,seed=1000,time=0.027461528778076172
done. name=original,Qubits=4, Gate_Num=5000, P_T=0.04,seed=1001,time=0.0276029109954834
done. name=original,Qubits=4, Gate_Num=5000, P_T=0.04,seed=1002,time=0.02882528305053711
done. name=original,Qubits=4, Gate_Num=5000, P_T=0.08,seed=1000,time=0.027564287185668945
done. name=original,Qubits=4, Gate_Num=5000, P_T=0.08,seed=1001,time=0.05792379379272461
done. name=original,Qubits=4, Gate_Num=5000, P_T=0.08,seed=1002,time=0.028658628463745117
done. name=original,Qubits=4, Gate_Num=5000, P_T=0.12,seed=1000,time=0.0293276309967041
done. name=original,Qubits=4, Gate_Num=5000, P_T=0.12,seed=1001,time=0.028191566467285156
done. name=original,